In [2]:
import numpy as np
import scipy as sc
import matplotlib as mpl
import matplotlib.pyplot as plt
from uncertainties import unumpy as unp
from uncertainties import ufloat, umath

from options import *
from constants import *

nom_vals = unp.nominal_values
std_devs = unp.std_devs

mpl.rcParams.update(rcParams)

In [3]:
N1, N2, N12, t = np.loadtxt("../data/activity_co57.csv", delimiter=',', unpack=True, skiprows=1)

N1 = unp.uarray(N1, np.sqrt(N1))
N2 = unp.uarray(N2, np.sqrt(N2))
N12 = unp.uarray(N12, np.sqrt(N12))
t = unp.uarray(t, ERROR_TIME)

m1 = N1 / t
m2 = N2 / t
m12 = N12 / t

In [4]:
twotheta = np.loadtxt("../data/2theta_resolution.csv")
twotheta

array(2.04202894e-06)

In [5]:
print(m1[0], m2[0], m12[0], twotheta)

7.68+/-0.05 465.1+/-0.4 0.873+/-0.016 2.042028935806682e-06


In [6]:
A = m1 * m2 / (m12 - twotheta * m1 * m2)
print(f"Best estimation for activity: {np.mean(A):.2upP} Bq")

Best estimation for activity: (4124±53) Bq


### Theory

In [8]:
import datetime as dt
now = dt.datetime.now()
prod = dt.datetime(year=2019, month=9, day=1)

t = ufloat((now - prod).total_seconds(), 14 * 24 * 60 * 60)
A0 = 366.8e3
lam = np.log(2) / (dt.timedelta(days=270).total_seconds())

est_A = A0 * unp.exp(-lam * t)
est_A

2995.1873011915645+/-107.64992171473475